In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w7-p2/sample_submission.csv
/kaggle/input/2021-ai-w7-p2/x_test.csv
/kaggle/input/2021-ai-w7-p2/y_train.csv
/kaggle/input/2021-ai-w7-p2/x_train.csv


In [2]:
x=pd.DataFrame(pd.read_csv("/kaggle/input/2021-ai-w7-p2/x_train.csv",engine='python',encoding='CP949'))
y=pd.DataFrame(pd.read_csv("/kaggle/input/2021-ai-w7-p2/y_train.csv",engine='python',encoding='CP949'))
test=pd.DataFrame(pd.read_csv("/kaggle/input/2021-ai-w7-p2/x_test.csv",engine='python',encoding='CP949'))
submit=pd.read_csv("/kaggle/input/2021-ai-w7-p2/sample_submission.csv")

In [3]:
y=y["교통량"]

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x)
x_std=scaler.transform(x)
test_std=scaler.transform(test)

In [5]:
import torch
import random
seed=777
random.seed(777)
torch.manual_seed(seed)
device='cuda' if torch.cuda.is_available() else 'cpu'
if device=='cuda':
  torch.cuda.manual_seed_all(seed)

In [6]:
import torch
x=np.array(x_std)
y=np.array(y)
test=np.array(test_std)
x=torch.FloatTensor(x)
y=torch.FloatTensor(y)
test=torch.FloatTensor(test)

In [7]:
linear1=torch.nn.Linear(14,128)
linear2=torch.nn.Linear(128,256)
linear3=torch.nn.Linear(256,64)
linear4=torch.nn.Linear(64,32)
linear5=torch.nn.Linear(32,1)

relu=torch.nn.ReLU()
dropout=torch.nn.Dropout(0.1)

In [8]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)


Parameter containing:
tensor([[ 0.3459,  0.1497, -0.0206, -0.2978,  0.0841,  0.1782, -0.1743,  0.2510,
          0.3327,  0.2294,  0.0673,  0.3381, -0.2895,  0.3038,  0.1379,  0.1645,
          0.0424,  0.4188, -0.2111, -0.2560,  0.0095,  0.1103,  0.1654, -0.0692,
          0.2748, -0.2246,  0.2565, -0.0802, -0.2142, -0.1114,  0.1294, -0.3466]],
       requires_grad=True)

In [9]:
model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4,relu,dropout,linear5).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss=torch.nn.MSELoss().to(device)

In [10]:
for stop in range(5000):
  optimizer.zero_grad()
  h=model(x.to(device))
  cost=loss(h,y.unsqueeze(1).to(device))
  cost.backward()
  optimizer.step()
  if stop%100==0:
    print(stop,cost.item())

0 422311.34375
100 221387.734375
200 182396.5625
300 164211.859375
400 126725.1015625
500 73649.4140625
600 45658.55859375
700 34085.1015625
800 28856.970703125
900 24406.181640625
1000 21737.134765625
1100 19986.625
1200 19175.0
1300 17866.201171875
1400 16437.365234375
1500 16017.755859375
1600 15623.998046875
1700 14951.7353515625
1800 13984.205078125
1900 13850.44140625
2000 13078.34765625
2100 13323.873046875
2200 12892.0
2300 12868.1015625
2400 12264.3876953125
2500 12203.6171875
2600 11787.5068359375
2700 11605.845703125
2800 11518.93359375
2900 11315.9326171875
3000 11274.87109375
3100 10887.7197265625
3200 11081.7939453125
3300 10935.125
3400 10905.685546875
3500 10373.255859375
3600 10553.9150390625
3700 10306.4140625
3800 10285.873046875
3900 10040.1064453125
4000 10045.296875
4100 10010.1767578125
4200 9624.66796875
4300 9876.4892578125
4400 9685.42578125
4500 9701.869140625
4600 9569.7919921875
4700 9375.30078125
4800 9156.943359375
4900 9363.328125


In [11]:
pred=list()
with torch.no_grad():
  model.eval()
  y_test=model(test.to(device))
  a,b=torch.max(y_test,dim=1)
  pred.extend(a.tolist())

In [12]:
submit["predict"]=pred
submit.to_csv("submit.csv",index=False)